In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import matching_script
import json

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

In [ ]:
#TODO, resolve difference between the old inventory and the old as some streets had their facilities upgraded recently (Edgewood) and (Confederate/United)
coa = gpd.read_file('D:/RAW/City_of_Atlanta/coa_bike_facilities_new.geojson')
coa.to_crs(config['projected_crs_epsg'],inplace=True)

Only keep funded infrastructure for now because the planned projects include ones that probably aren't happening soon. Have a talk with Rebecca Serna about which facilities in this plan to include.

In [ ]:
# only keep funded ones for now
coa = coa[(coa['Status']=='Funded')]

In [ ]:
# remove uncessary columns
coa.drop(columns=['GlobalID','Shape__Length','LengthMi','CrossSectionNotes','Status','YearInstalled'],inplace=True)

#rename the id/year column
coa.rename(columns={'OBJECTID':'id'},inplace=True)

# rename columns for consistency
coa.columns = ['coa_'+col.lower() if col != 'geometry' else 'geometry' for col in coa.columns.tolist()]

# convert facility type to OSM (use highest protection if two different types)
osm_types = ['sharrow','bike lane','buffered bike lane','cycletrack','multi use path']
coa_conversion = {
       'Protected Bike Lane': osm_types[3], 
       'Protected Bike Lane / Bike Lane': osm_types[3],
       'Two-Way Cycle Track': osm_types[3], 
       'Uphill Bike Lane / Downhill Sharrows': osm_types[1],
       'Sharrows': osm_types[0], 
       'Bike Lane': osm_types[1], 
       'Bike Lane ': osm_types[1],
       'Uphill Buffered Bike Lane / Downhill Sharrows': osm_types[2],
       'Buffered Bike Lane': osm_types[2], 
       'Buffered Contraflow Bike Lane / Bike Lane': osm_types[1],
       'Shared-Use Path': osm_types[4], 
       'Neighborhood Greenway': osm_types[0], 
       'Bike Lane / Sharrows': osm_types[1],
       'Shared-Use Path / Bike Lane': osm_types[4], 
       'Buffered Bike Lane / Bike Lane': osm_types[2],
       'Buffered Bike Lane / Shared-Use Path': osm_types[4],
       'Shared-Use Path / Sharrows': osm_types[4],
       'Uphill Protected Bike Lane / Downhill Sharrows': osm_types[3], 
       'Shared Path': osm_types[4],
       'Seperated Bike Lane': osm_types[3],
       'Contraflow Bike Lane': osm_types[1], 
       'Bike Boulevard': osm_types[0],
       'Contraflow Bike Lane / Sharrows': osm_types[2], 
       'Separated Bike Lane / Buffered Bike Lane': osm_types[3],
       'Separated Bike Lane / Bike Lane': osm_types[3], 
       'Separated Bike Lane': osm_types[3],
       'Neighborhood Greenway / Bike Lane': osm_types[1], 
       'Buffered Bike Lanes': osm_types[2],
       'Raised Bike Lane': osm_types[3], 
}
coa['coa_osm_type'] = coa['coa_facilitytype'].map(coa_conversion)

In [ ]:
# remove sharrows and nas
coa = coa[(coa['coa_osm_type'] != 'sharrow') & coa['coa_osm_type'].notna()]

In [ ]:
coa['coa_osm_type'].value_counts(dropna=False)

In [ ]:
coa.head()

In [ ]:
#export
coa.to_file(config['bicycle_facilities_fp']/'network_modifications.gpkg',layer='coa')

In [ ]:
# m = coa.explore('coa_osm_type',tiles="CartoDBpositron")
# m